<a href="https://colab.research.google.com/github/harsh-sengar/tuberculosis-drug-resistance-classification/blob/master/tuberculosisMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/My Drive/IIT Kanpur/Genomics/Assignment1/data/X_trainData_1.csv"
X_trainData_1 = pd.read_csv(path)

path1 = "/content/drive/My Drive/IIT Kanpur/Genomics/Assignment1/data/Y_trainData_1.csv"
Y_trainData_1 = pd.read_csv(path1)

In [5]:
x_train1 = np.array(X_trainData_1)
y_train1 = np.array(Y_trainData_1)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x_train1, y_train1, test_size=0.33, random_state=42)

# Xgboost classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

clf = MultiOutputClassifier(XGBClassifier(max_depth=7, eta = 0.2, gamma = 3, nrounds = 40))

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#clf.score(X_test, y_test)

## Neural Network for Single Task Classification

In [8]:
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

X = x_train1
Y = y_train1[:,0]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
encoded_Y = np_utils.to_categorical(encoded_Y)

def neuralNetworkModelWithTHREEoutputs():
    model = Sequential()
    model.add(Dense(200, input_dim = 222, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(75, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(3, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=neuralNetworkModelWithTHREEoutputs, epochs=20, batch_size=50, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("neuralNetwork: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



neuralNetwork: 95.05% (1.46%)
